In [2]:
import torch
import sys
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
import training
import config
import matplotlib.pyplot as plt
import os

In [9]:
results = torch.zeros([10,3,6,3,2])

for seed in range(10):
    for l,lr in enumerate([-4,-5,-6]):
        for n,num_layer in enumerate([7,8,9]):
            for o,order in enumerate([3,4,5,6,7,8]):
                data = torch.load(f'./data/dataset_order_{order}.ds')
                Xn_valid = data['X_valid']
                Yn_valid = data['Y_valid']
                Xn_test  = data['X_test']
                Yn_test  = data['Y_test']

                valid_data = TensorDataset(Xn_valid, Yn_valid)
                test_data  = TensorDataset(Xn_test, Yn_test)
                valid_loader = DataLoader(valid_data, batch_size=len(valid_data))
                test_loader  = DataLoader(test_data, batch_size=len(test_data))

                modelname = f'predictor_{num_layer}_{order}_{lr}_{seed}'

                if os.path.isfile(f'./NNs/{modelname}'):

                    model = torch.load(f'./NNs/{modelname}')

                    config.SetSeed(seed)
                    for x,y in valid_loader:
                        X_valid, y_valid = x, y
                    for x,y in test_loader:
                        X_test, y_test = x, y

                    loss_fn = torch.nn.MSELoss(reduction='mean')
                    loss_valid = loss_fn(model(X_valid), y_valid)
                    loss_test  = loss_fn(model(X_test),  y_test)

                    results[seed, n, o, l, 0] = loss_valid
                    results[seed, n, o, l, 1] = loss_test
                    print(loss_valid, loss_test)
                else:
                    results[seed, n, o, l, :] = float('nan')

tensor(0.0003, grad_fn=<MseLossBackward0>) tensor(0.0003, grad_fn=<MseLossBackward0>)
tensor(0.0078, grad_fn=<MseLossBackward0>) tensor(0.0077, grad_fn=<MseLossBackward0>)
tensor(7.2002e-05, grad_fn=<MseLossBackward0>) tensor(7.2197e-05, grad_fn=<MseLossBackward0>)
tensor(5.5968e-05, grad_fn=<MseLossBackward0>) tensor(5.5805e-05, grad_fn=<MseLossBackward0>)
tensor(4.0153e-05, grad_fn=<MseLossBackward0>) tensor(4.2404e-05, grad_fn=<MseLossBackward0>)
tensor(4.4146e-05, grad_fn=<MseLossBackward0>) tensor(4.5551e-05, grad_fn=<MseLossBackward0>)
tensor(0.0001, grad_fn=<MseLossBackward0>) tensor(0.0001, grad_fn=<MseLossBackward0>)
tensor(8.4950e-05, grad_fn=<MseLossBackward0>) tensor(0.0001, grad_fn=<MseLossBackward0>)
tensor(2.8461e-05, grad_fn=<MseLossBackward0>) tensor(2.6307e-05, grad_fn=<MseLossBackward0>)
tensor(4.1055e-05, grad_fn=<MseLossBackward0>) tensor(4.2402e-05, grad_fn=<MseLossBackward0>)
tensor(0.0002, grad_fn=<MseLossBackward0>) tensor(0.0001, grad_fn=<MseLossBackward0>)
te

In [12]:
results_mean_seed = torch.tensor(np.nanmean(results.detach().numpy(), axis=0)) # [layer, order, lr, loss]
results_mean_seed.shape

/var/folders/j1/dh3xq5ns6xlbwp075yd1vt_00000gn/T/ipykernel_31708/2923597048.py:1: RuntimeWarning: Mean of empty slice
  results_mean_seed = torch.tensor(np.nanmean(results.detach().numpy(), axis=0)) # [layer, order, lr, loss]


torch.Size([3, 6, 3, 2])

In [17]:
results_mean_seed = results_mean_seed[:,:,0,:] # [layer, order, loss]
results_mean_seed.shape

torch.Size([3, 6, 2])

In [18]:
# layer
results_layer = results_mean_seed.reshape(3,-1,2).min(1)[0]
results_layer

tensor([[5.9000e-05, 5.9503e-05],
        [9.9725e-05, 9.6080e-05],
        [2.1549e-05, 2.3785e-05]])

In [19]:
# order
results_layer = results_mean_seed.permute(1,0,2).reshape(6,-1,2).min(1)[0]
results_layer

tensor([[1.6777e-04, 1.6513e-04],
        [1.2196e-04, 1.2713e-04],
        [1.5576e-04, 1.5081e-04],
        [9.8651e-05, 9.8602e-05],
        [2.1549e-05, 2.3785e-05],
        [3.8960e-05, 4.1064e-05]])